In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tqdm import tqdm_notebook

In [18]:
df = pd.read_csv('data/HousePrice_preprocessed_df2_wSalePrice_test3.csv')
df

,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,SalePrice
0,0.575228,0.125000,-0.288554,-0.944267,208500
1,1.171591,0.125000,-0.288554,-0.641008,181500
2,0.092875,0.125000,-0.288554,-0.301540,223500
3,-0.499103,0.125000,-0.288554,-0.061648,140000
4,0.463410,0.125000,-0.288554,-0.174805,250000
5,0.632233,0.125000,-0.288554,-1.138896,143000
6,2.028862,0.125000,-0.288554,-0.566325,307000
7,0.910682,0.583333,-0.090190,-0.794901,200000
8,-0.972685,0.125000,-0.288554,0.870759,129900
9,0.893142,0.125000,-0.288554,-0.966898,118000


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
BsmtFinSF1      10 non-null float64
BsmtFinType2    10 non-null float64
BsmtFinSF2      10 non-null float64
BsmtUnfSF       10 non-null float64
SalePrice       10 non-null int64
dtypes: float64(4), int64(1)
memory usage: 480.0 bytes


In [25]:
tf.set_random_seed(123)
filename_queue = tf.train.string_input_producer(['./data/HousePrice_preprocessed_df2_wSalePrice.csv'], 
                                                shuffle=False, name='filename_queue')
key, value = tf.TextLineReader(skip_header_lines=1).read(filename_queue)
record_defaults = [[0.], [0.],[0.], [0.],[0.], [0.],[0.], [0.],[0.], [0.],
                  [0.], [0.],[0.], [0.],[0.], [0.],[0.], [0.],[0.], [0.],
                  [0.], [0.],[0.], [0.],[0.], [0.],[0.]] # 27칸
xy = tf.decode_csv(value, record_defaults=record_defaults)

# collect batches of csv in
train_x_batch, train_y_batch = tf.train.batch([xy[:-1], xy[-1:]], batch_size=1)

X = tf.placeholder(tf.float32, shape=[None, 26])
Y = tf.placeholder(tf.float32, shape=[None, 1])
W = tf.Variable(tf.random_normal([26,1], name = 'weight'))
b = tf.Variable(tf.random_normal([1], name='bias'))

hypothesis = tf.matmul(X, W) + b
cost = tf.reduce_mean(tf.square(hypothesis - Y))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(cost)

In [26]:
# tf.set_random_seed(123)
# filename_queue = tf.train.string_input_producer(['./data/HousePrice_preprocessed_df2_wSalePrice.csv'], 
#                                                 shuffle=False, name='filename_queue')
# key, value = tf.TextLineReader(skip_header_lines=1).read(filename_queue)
# record_defaults = [tf.constant([], dtype=tf.float32),    # Column 0
#                    tf.constant([], dtype=tf.float32),    # Column 1
#                    tf.constant([], dtype=tf.float32),    # Column 2
#                    tf.constant([], dtype=tf.float32),
#                   tf.constant([], dtype=tf.int64)] # 27칸
# xy = tf.decode_csv(value, record_defaults=record_defaults)

# # collect batches of csv in
# train_x_batch, train_y_batch = tf.train.batch([xy[:-1], xy[-1:]], batch_size=1)

# X = tf.placeholder(tf.float32, shape=[None, 4])
# Y = tf.placeholder(tf.float32, shape=[None, 1])
# W = tf.Variable(tf.random_normal([4,1], name = 'weight'))
# b = tf.Variable(tf.random_normal([1], name='bias'))

# hypothesis = tf.matmul(X, W) + b
# cost = tf.reduce_mean(tf.square(hypothesis - Y))

# optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
# train = optimizer.minimize(cost)

In [27]:
train_x_batch.shape, train_y_batch.shape

(TensorShape([Dimension(1), Dimension(26)]),
 TensorShape([Dimension(1), Dimension(1)]))

In [28]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())
# print(sess.run(W), sess.run(b))

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

Step_val = []
Cost_val = []

for step in tqdm_notebook(range(4001)):
    x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
    cost_val, hy_val, _ = sess.run([cost, hypothesis, train], 
                                   feed_dict={X: x_batch,
                                              Y: y_batch})
    Step_val.append(step)
    Cost_val.append(cost_val)

    if step % 100 == 0 or step < 10:            
        print('step {}: \nhy_val: {}, \ncost_val: {}'.format(step, hy_val, cost_val))
coord.request_stop()
coord.join(threads)

step 0: 
hy_val: [[-2.9278107]], 
cost_val: 43473469440.0
step 1: 
hy_val: [[25779.66]], 
cost_val: 24248825856.0
step 2: 
hy_val: [[75653.79]], 
cost_val: 21858504704.0
step 3: 
hy_val: [[105710.31]], 
cost_val: 1175782656.0
step 4: 
hy_val: [[133573.08]], 
cost_val: 13555227648.0
step 5: 
hy_val: [[175735.39]], 
cost_val: 1071605824.0
step 6: 
hy_val: [[122970.28]], 
cost_val: 33866938368.0
step 7: 
hy_val: [[218146.02]], 
cost_val: 329277888.0
step 8: 
hy_val: [[129647.67]], 
cost_val: 63669.484375
step 9: 
hy_val: [[136675.38]], 
cost_val: 348769632.0
step 100: 
hy_val: [[179945.16]], 
cost_val: 627745216.0
step 200: 
hy_val: [[128913.08]], 
cost_val: 122919840.0
step 300: 
hy_val: [[158259.84]], 
cost_val: 1587206.25
step 400: 
hy_val: [[177795.8]], 
cost_val: 4583859200.0
step 500: 
hy_val: [[136730.92]], 
cost_val: 563156672.0
step 600: 
hy_val: [[290368.53]], 
cost_val: 236191760.0
step 700: 
hy_val: [[394701.53]], 
cost_val: 6757091840.0
step 800: 
hy_val: [[189147.58]], 
cost

In [9]:
# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

# Start populating the filename queue.
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

Step_val = []
Cost_val = []

for step in tqdm_notebook(range(2001)):

    x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
    cost_val, hy_val, _ = sess.run(
        [cost, hypothesis, train], feed_dict={X: x_batch, Y: y_batch})
    

    Step_val.append(step)
    Cost_val.append(cost_val)
    
    if step % 100 == 0 or step < 10 :
        print("\nStep : {} \nCost : {} \nPrediction :\n{}".format(step, cost_val, hy_val))

coord.request_stop()
coord.join(threads)

INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>, Field 0 in record 0 is not a valid float: Foundation
	 [[Node: DecodeCSV = DecodeCSV[OUT_TYPE=[DT_FLOAT, DT_FLOAT, DT_FLOAT], field_delim=",", na_value="", use_quote_delim=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](ReaderReadV2:1, DecodeCSV/record_defaults_0, DecodeCSV/record_defaults_0, DecodeCSV/record_defaults_0)]]


OutOfRangeError: FIFOQueue '_10_batch_1/fifo_queue' is closed and has insufficient elements (requested 1, current size 0)
	 [[Node: batch_1 = QueueDequeueManyV2[component_types=[DT_FLOAT, DT_FLOAT], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/device:CPU:0"](batch_1/fifo_queue, batch_1/n)]]

Caused by op 'batch_1', defined at:
  File "C:\Python\Anaconda3-52\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Python\Anaconda3-52\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Python\Anaconda3-52\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Python\Anaconda3-52\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Python\Anaconda3-52\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
  File "C:\Python\Anaconda3-52\lib\site-packages\tornado\platform\asyncio.py", line 132, 